SparkMLlib - ALS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error
import gzip
import json
from copy import deepcopy

Read Data

In [ ]:
data1= []
with gzip.open(r'/content/meta_Luxury_Beauty.json.gz') as f:
    for l in f:
        data1.append(json.loads(l.strip()))
print(len(data1))
print(data1[0])

12299
{'category': [], 'tech1': '', 'description': ['After a long day of handling thorny situations, our new hand therapy pump is just the help you need. It contains shea butter as well as extracts of yarrow, clover and calendula to help soothe and condition work-roughened hands.', 'By Crabtree & Evelyn', 'The aromatic benefits of herbs are varied and far-reaching, so we combined a whole bunch of them into one restoratively fragrant line-up straight from the garden.', 'We&#039;ve formulated our Gardeners Hand Therapy with Myrrh Extract to help condition nails and cuticles as well as skin super hydrators macadamia seed oil and shea butter to help replenish lost moisture. Rich in herbal extracts like cooling cucumber and rosemary leaf  a favourite for antioxidants  to help protect hands against daily urban and environmental stresses while the hydrating power of Vitamin E, Hyaluronic Acid and Ceramides contribute to improve the skins natural moisture barrier with this garden-inspired trea

In [ ]:
df1 = pd.DataFrame.from_dict(data1)
print(len(df1))

12299


In [ ]:
df1

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[After a long day of handling thorny situation...,,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"[B00GHX7H0A, B00FRERO7G, B00R68QXCS, B000Z65AZ...",,,[],"4,324 in Beauty & Personal Care (","[B00FRERO7G, B00GHX7H0A, B07GFHJRMX, B00TJ3NBN...",{' Product Dimensions: ': '2.2 x 2.2 ...,Luxury Beauty,,,$30.00,B00004U9V2,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[If you haven't experienced the pleasures of b...,,AHAVA Bath Salts,[],,,[],"1,633,549 in Beauty & Personal Care (",[],{' Product Dimensions: ': '3 x 3.5 x ...,Luxury Beauty,,,,B0000531EN,[],[]
2,[],,"[Rich, black mineral mud, harvested from the b...",,"AHAVA Dead Sea Mineral Mud, 8.5 oz, Pack of 4",[],,,[],"1,806,710 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '5.1 x 3 x ...,Luxury Beauty,,,,B0000532JH,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[This liquid soap with convenient pump dispens...,,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",[],,,[],[],"[B00004U9V2, B00GHX7H0A, B00FRERO7G, B00R68QXC...",{' Product Dimensions: ': '2.6 x 2.6 ...,Luxury Beauty,,,$15.99,B00005A77F,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Remember why you love your favorite blanket? ...,,Soy Milk Hand Crme,"[B000NZT6KM, B001BY229Q, B008J724QY, B0009YGKJ...",,,[],"42,464 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '7.2 x 2.2 ...,Luxury Beauty,,,$18.00,B00005NDTD,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12294,[],,"[, CND Craft Culture Collection: Patina Buckle...",,"CND Shellac Power Polish, Patina Buckle","[B003ONLAXQ, B00YDEZ9T6, B074KHRD13, B00R3PZK1...",,,[],"88,740 in Beauty & Personal Care (","[B00D2VMUA2, B074KJZJYW, B074KHRD13, B073SB9JW...","{' Item Weight: ': '0.48 ounces', 'Sh...",Luxury Beauty,,,$15.95,B01HIQIEYC,[],[]
12295,[],,[CND Shellac was designed to be used as a syst...,,CND Shellac power polish denim patch,"[B003ONLAXQ, B003OH0KBA, B004LEMWGG, B01MT91G4...",,,[],"122,331 in Beauty & Personal Care (","[B00D2VMUA2, B01L0EV8X2, B004LEMWGG, B00EFGDYZ...","{'Shipping Weight:': '1.4 ounces (', 'ASIN:': ...",Luxury Beauty,,,$15.95,B01HIQHQU0,[],[]
12296,[],,[CND Shellac was designed to be used as a syst...,,"CND Shellac, Leather Satchel","[B003ONLAXQ, B003OH0KBA, B004LEMWGG, B01MT91G4...",,,[],"168,028 in Beauty & Personal Care (","[B00D2VMUA2, B01L0EV8X2, B004LEMWGG, B00EFGDYZ...","{'Shipping Weight:': '1.4 ounces (', 'Domestic...",Luxury Beauty,,,$15.95,B01HIQEOLO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
12297,[],,[The I AM JUICY COUTURE girl is once again tak...,,"Juicy Couture I Love Juicy Couture, 1.7 fl. Oz...",[],,,[],"490,755 in Beauty & Personal Care (","[B0757439SY, B01HJ2UY1G, B01KX3TK7C, B01LX71LJ...",{' Product Dimensions: ': '3.3 x 2.7 ...,Luxury Beauty,,,$76.00,B01HJ2UY0W,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
df1.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [ ]:
df1.drop([ 'tech1', 'description', 'fit', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'imageURL', 'imageURLHighRes', 'details'], axis=1, inplace=True)

In [ ]:
df1

,category,title,asin
0,[],HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,0030672120
1,[],"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",0071480935
2,[],Connect with LearnSmart Access Card for Microb...,007329506X
3,[],LearnSmart Standalone Access Card for Prescott...,0073513458
4,[],Anatomy &amp; Physiology Revealed Student Acce...,0073525758
...,...,...,...
26785,"[Software, Children's]",925 Sterling Silver Angel Wings Heart Pendant ...,B01HEFZJC2
26786,"[Software, Children's, Material: Guaranteed 10...",925 Sterling Silver Love Heart Opal Pendant Ne...,B01HEFZKEE
26787,"[Software, Digital Software, Antivirus & Secur...",Intego Mac Internet Security X9 - 1 Mac - 1 ye...,B01HF3G4BS
26788,[],VersaCheck X9 Small and Medium Business 2016 3...,B01HF41TKI


In [ ]:
meta_list=df1.values.tolist()
meta_list

[[[], 'HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK CONCEPTS', '0030672120'],
 [[],
  'Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish',
  '0071480935'],
 [[],
  'Connect with LearnSmart Access Card for Microbiology Fundamentals: A Clinical Approach',
  '007329506X'],
 [[],
  "LearnSmart Standalone Access Card for Prescott's Microbiology 9e",
  '0073513458'],
 [[],
  'Anatomy &amp; Physiology Revealed Student Access Card (Cat Version)',
  '0073525758'],
 [[],
  'Connect Math by ALEKS Access Card 52 Weeks for College Algebra Essentials',
  '0077340701'],
 [[],
  'Connect Access Card for Microbiology Fundamentals: A Clinical Approach',
  '0077369823'],
 [[],
  'Connect Art with McGraw-Hill LearnSmart Access Card for Living with Art',
  '0077410297'],
 [[],
  'Connect Biology Access Card for Essentials of the Living World',
  '007739688X'],
 [[],
  'Connect Art PLUS with McGraw-Hill LearnSmart Access Card for Living with Art  (Access

In [ ]:
data2 = []
with gzip.open(r'/content/meta_Software.json.gz') as f:
    for l in f:
        data2.append(json.loads(l.strip()))
print(len(data2))
print(data2[0])

883636
{'overall': 5.0, 'verified': True, 'reviewTime': '10 20, 2014', 'reviewerID': 'A1D4G1SNUZWQOT', 'asin': '7106116521', 'reviewerName': 'Tracy', 'reviewText': 'Exactly what I needed.', 'summary': 'perfect replacements!!', 'unixReviewTime': 1413763200}


In [ ]:
df2 = pd.DataFrame.from_dict(data2)
print(len(df2))

883636


In [ ]:
df2.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

##preprocess

In [ ]:
df2.drop(['verified', 'reviewTime', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'], axis = 1, inplace = True)

In [ ]:
df2

,overall,reviewerID,asin
0,5.0,A1D4G1SNUZWQOT,7106116521
1,2.0,A3DDWDH9PX2YX2,7106116521
2,4.0,A2MWC41EW7XL15,7106116521
3,2.0,A2UH2QQ275NV45,7106116521
4,3.0,A89F3LQADZBS5,7106116521
...,...,...,...
883631,5.0,A1ZSB2Q144UTEY,B01HJHTH5U
883632,5.0,A2CCDV0J5VB6F2,B01HJHTH5U
883633,3.0,A3O90PACS7B61K,B01HJHTH5U
883634,3.0,A2HO94I89U3LNH,B01HJHF97K


In [ ]:
df2 = df2[:10000]

In [ ]:
from copy import deepcopy
Letssee=deepcopy(df2)

In [ ]:
selected_columns = df2[["reviewerID","asin","overall"]]
new_df = selected_columns.copy()
print(new_df)

          reviewerID        asin  overall
0     A1D4G1SNUZWQOT  7106116521      5.0
1     A3DDWDH9PX2YX2  7106116521      2.0
2     A2MWC41EW7XL15  7106116521      4.0
3     A2UH2QQ275NV45  7106116521      2.0
4      A89F3LQADZBS5  7106116521      3.0
...              ...         ...      ...
9995  A30X3UYW2PGPT0  B000JOOR7O      5.0
9996  A2EHBC3JYMPZ7G  B000JOOR7O      5.0
9997  A2P8ZC2J02FB4I  B000JOOR7O      2.0
9998  A2G0FIQB2KLULZ  B000JOOR7O      5.0
9999  A14WK4K3M8YJ7F  B000JOOR7O      5.0

[10000 rows x 3 columns]


In [ ]:
unique_asin = new_df['asin'].unique()
count = 0
asin_to_num_dict = {}
num_to_asin_dict = {}
for i in unique_asin:
    asin_to_num_dict[i] = count
    num_to_asin_dict[count] = i
    count+=1

In [ ]:
lol = new_df.values.tolist()
lol

[['A1D4G1SNUZWQOT', '7106116521', 5.0],
 ['A3DDWDH9PX2YX2', '7106116521', 2.0],
 ['A2MWC41EW7XL15', '7106116521', 4.0],
 ['A2UH2QQ275NV45', '7106116521', 2.0],
 ['A89F3LQADZBS5', '7106116521', 3.0],
 ['A29HLOUW0NS0EH', '7106116521', 5.0],
 ['A7QS961ROI6E0', '7106116521', 4.0],
 ['A1BB77SEBQT8VX', 'B00007GDFV', 3.0],
 ['AHWOW7D1ABO9C', 'B00007GDFV', 3.0],
 ['AKS3GULZE0HFC', 'B00007GDFV', 3.0],
 ['A38NS6NF6WPXS', 'B00007GDFV', 4.0],
 ['A1KOKO3HTSAI1H', 'B00007GDFV', 2.0],
 ['A1G3S57JGZNPCL', 'B00007GDFV', 1.0],
 ['AGBL3TTP6GV4X', 'B00007GDFV', 1.0],
 ['A1Y36BSE9GKXLV', 'B00007GDFV', 4.0],
 ['A1L1U968VNYVA4', 'B00007GDFV', 3.0],
 ['A1NSKPSR0XZ0C9', 'B00007GDFV', 5.0],
 ['A3O5SXH5O8DWRP', 'B00007GDFV', 5.0],
 ['A3I52T3ZCLRZZA', 'B00007GDFV', 4.0],
 ['A3VWTJR1QOI7JR', 'B00007GDFV', 3.0],
 ['A1L5PG0YZ4TNSZ', 'B00007GDFV', 5.0],
 ['A19OTBIPAMJUU1', 'B00007GDFV', 1.0],
 ['A3IT0MD9QUFZEK', 'B00007GDFV', 4.0],
 ['A1IER9LE7FBJ95', 'B00007GDFV', 4.0],
 ['A3DZQ7TB4ZUWKG', 'B00007GDFV', 5.0],
 ['A19

In [ ]:
l=[]
for i in asin_to_num_dict.keys():
    l.append([i,asin_to_num_dict[i]])
dff=pd.DataFrame(l)

In [ ]:
dff.rename({1:'asin2num',0:'asin'}, axis=1,inplace=True)

In [ ]:
df_inner = pd.merge(new_df,dff, on='asin', how='inner')

df_inner

,reviewerID,asin,overall,asin2num
0,A1D4G1SNUZWQOT,7106116521,5.0,0
1,A3DDWDH9PX2YX2,7106116521,2.0,0
2,A2MWC41EW7XL15,7106116521,4.0,0
3,A2UH2QQ275NV45,7106116521,2.0,0
4,A89F3LQADZBS5,7106116521,3.0,0
...,...,...,...,...
9995,A2EHBC3JYMPZ7G,B000JOOR7O,5.0,106
9996,A2P8ZC2J02FB4I,B000JOOR7O,2.0,106
9997,A2G0FIQB2KLULZ,B000JOOR7O,5.0,106
9998,A14WK4K3M8YJ7F,B000JOOR7O,5.0,106


In [ ]:
unique_reviewer = new_df['reviewerID'].unique()
count = 0
reviewer_to_num_dict = {}
num_to_reviewer_dict = {}
for i in unique_reviewer:
    reviewer_to_num_dict[i] = count
    num_to_reviewer_dict[count] = i
    count+=1

In [ ]:
l=[]
for i in reviewer_to_num_dict.keys():
    l.append([i,reviewer_to_num_dict[i]])
dff2=pd.DataFrame(l)

In [ ]:
dff2.rename({1:'reviewer2num',0:'reviewerID'}, axis=1,inplace=True)

In [ ]:
df_inner2 = pd.merge(df_inner,dff2, on='reviewerID', how='inner')

df_inner2

,reviewerID,asin,overall,asin2num,reviewer2num
0,A1D4G1SNUZWQOT,7106116521,5.0,0,0
1,A3DDWDH9PX2YX2,7106116521,2.0,0,1
2,A2MWC41EW7XL15,7106116521,4.0,0,2
3,A2UH2QQ275NV45,7106116521,2.0,0,3
4,A89F3LQADZBS5,7106116521,3.0,0,4
...,...,...,...,...,...
9995,A2EHBC3JYMPZ7G,B000JOOR7O,5.0,106,8549
9996,A2P8ZC2J02FB4I,B000JOOR7O,2.0,106,8550
9997,A2G0FIQB2KLULZ,B000JOOR7O,5.0,106,8551
9998,A14WK4K3M8YJ7F,B000JOOR7O,5.0,106,8552


In [ ]:
df_final=df_inner2.drop(['reviewerID','asin'],axis=1)

In [ ]:
df_final

,overall,asin2num,reviewer2num
0,5.0,0,0
1,2.0,0,1
2,4.0,0,2
3,2.0,0,3
4,3.0,0,4
...,...,...,...
9995,5.0,106,8549
9996,2.0,106,8550
9997,5.0,106,8551
9998,5.0,106,8552


In [ ]:
selected_columns = df_final[["reviewer2num","asin2num","overall"]]
new_df_final = selected_columns.copy()
print(new_df_final)

      reviewer2num  asin2num  overall
0                0         0      5.0
1                1         0      2.0
2                2         0      4.0
3                3         0      2.0
4                4         0      3.0
...            ...       ...      ...
9995          8549       106      5.0
9996          8550       106      2.0
9997          8551       106      5.0
9998          8552       106      5.0
9999          8496       107      4.0

[10000 rows x 3 columns]


In [ ]:
from copy import deepcopy

In [ ]:
df_l2=deepcopy(new_df_final)

In [ ]:
df_l2.drop_duplicates(subset=["reviewer2num","asin2num"], keep='first', inplace=True)

In [ ]:
df_mat = df_l2.pivot(index='reviewer2num', columns='asin2num', values='overall')
df_mat

asin2num,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
reviewer2num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
8549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
8550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN


In [ ]:
df_mat.fillna(0)

asin2num,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
reviewer2num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
8549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
8550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
df_mat2=df_mat.fillna(0)

In [ ]:
df_mat2

asin2num,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
reviewer2num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
8549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
8550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
ratings=csr_matrix(df_mat2)

In [ ]:
print(ratings)

  (0, 0)	5.0
  (1, 0)	2.0
  (2, 0)	4.0
  (3, 0)	2.0
  (4, 0)	3.0
  (5, 0)	5.0
  (6, 0)	4.0
  (7, 1)	3.0
  (8, 1)	3.0
  (9, 1)	3.0
  (10, 1)	4.0
  (11, 1)	2.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 1)	4.0
  (15, 1)	3.0
  (16, 1)	5.0
  (17, 1)	5.0
  (18, 1)	4.0
  (19, 1)	3.0
  (20, 1)	5.0
  (21, 1)	1.0
  (22, 1)	4.0
  (23, 1)	4.0
  (24, 1)	5.0
  :	:
  (8528, 106)	5.0
  (8529, 106)	5.0
  (8530, 106)	5.0
  (8531, 106)	3.0
  (8532, 106)	2.0
  (8533, 106)	5.0
  (8534, 106)	4.0
  (8535, 106)	5.0
  (8536, 106)	3.0
  (8537, 106)	5.0
  (8538, 106)	5.0
  (8539, 106)	5.0
  (8540, 106)	4.0
  (8541, 106)	5.0
  (8542, 106)	5.0
  (8543, 106)	5.0
  (8544, 106)	4.0
  (8545, 106)	5.0
  (8546, 106)	5.0
  (8547, 106)	5.0
  (8548, 106)	5.0
  (8549, 106)	5.0
  (8550, 106)	2.0
  (8551, 106)	5.0
  (8552, 106)	5.0


##recommendation

In [ ]:
pip install implicit

     |████████████████████████████████| 1.1 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4503541 sha256=17723e2442ae3e6238ebe3d93de0a0c27a1c8d6fc9067ceae33fd971332ae701
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
model = implicit.als.AlternatingLeastSquares(factors=50)
#model = implicit.bpr.BayesianPersonalizedRanking(factors=128)

model.fit(ratings) #rating  =  sparse matrix

# recommend items for a user
user_items = ratings.T.tocsr()
recommendations = model.recommend(1, user_items)

related = model.similar_items(1)

#print(recommendations)
#print(related)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
related

[(0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0),
 (0, -0.0)]

In [ ]:
recommendations

[]

In [ ]:
num_to_asin_dict

In [ ]:
n=len(related)
l=[]
for i in range(n):
    s=[]
    for j in num_to_asin_dict.keys():
        if i==j:
            s.append(num_to_asin_dict[j])
            s.append(related[i][1])
    l.append(s)
print(l)

In [ ]:
result = pd.DataFrame(l)
result

In [ ]:
result.rename({0:'asin',1:'accuracy'}, axis=1,inplace=True)

In [ ]:
result

,asin,accuracy
0,7106116521,1.000000
1,B00007GDFV,0.842161
2,B00008JOQI,0.834434
3,B00008JQF0,0.828605
4,B00008JVTT,0.826245
5,B00008JPRZ,0.823265
6,B0000AWHAA,0.822986
7,B0000AWXMM,0.819776
8,B0000AWX3V,0.819721
9,B00015VKT4,0.818363


In [ ]:
df1

,title,asin
0,Slime Time Fall Fest [With CDROM and Collector...,0764443682
1,XCC Qi promise new spider snake preparing men'...,1291691480
2,Magical Things I Really Do Do Too!,1940280001
3,"Ashes to Ashes, Oranges to Oranges",1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,1940967805
...,...,...
186632,JT Women's Elegant Off Shoulder Chiffon Maxi L...,B01HJGXL4O
186633,Microcosm Retro Vintage Black Crochet Lace One...,B01HJHF97K
186634,Lookatool Classic Plain Vintage Army Military ...,B01HJGJ9LS
186635,Edith Windsor Women's Deep V-neck Beaded Sequi...,B01HJHTH5U


In [ ]:
recommended = pd.merge(result,df1, on='asin', how='inner')

recommended

,asin,accuracy,title
0,7106116521,1.000000,Milliongadgets(TM) Earring Safety Backs For Fi...
1,B00007GDFV,0.842161,Buxton Heiress Pik-Me-Up Framed Case
2,B00007GDFV,0.842161,Buxton Heiress Pik-Me-Up Framed Case
3,B00008JOQI,0.834434,Paul Fredrick Men's Pinpoint Button Down Colla...
4,B00008JOQI,0.834434,Paul Fredrick Men's Pinpoint Button Down Colla...
5,B00008JQF0,0.828605,Paul Fredrick Men's Pinpoint Straight Collar B...
6,B00008JQF0,0.828605,Paul Fredrick Men's Pinpoint Straight Collar B...
7,B00008JVTT,0.826245,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
8,B00008JVTT,0.826245,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
9,B00008JPRZ,0.823265,Paul Fredrick Men's Pinpoint Snap Tab Collar B...


In [ ]:
n=len(recommendations)
l=[]
for i in range(n):
    s=[]
    for j in num_to_asin_dict.keys():
        if i==j:
            s.append(num_to_asin_dict[j])
            s.append(recommendations[i][1])
    l.append(s)
result = pd.DataFrame(l)
result.rename({0:'asin',1:'accuracy'}, axis=1,inplace=True)
user_recommended = pd.merge(result,df1, on='asin', how='inner')

user_recommended


,asin,accuracy,title
0,7106116521,0.003633,Milliongadgets(TM) Earring Safety Backs For Fi...
1,B00007GDFV,0.003627,Buxton Heiress Pik-Me-Up Framed Case
2,B00007GDFV,0.003627,Buxton Heiress Pik-Me-Up Framed Case
3,B00008JOQI,0.003623,Paul Fredrick Men's Pinpoint Button Down Colla...
4,B00008JOQI,0.003623,Paul Fredrick Men's Pinpoint Button Down Colla...
5,B00008JQF0,0.003584,Paul Fredrick Men's Pinpoint Straight Collar B...
6,B00008JQF0,0.003584,Paul Fredrick Men's Pinpoint Straight Collar B...
7,B00008JVTT,0.003582,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
8,B00008JVTT,0.003582,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
9,B00008JPRZ,0.003582,Paul Fredrick Men's Pinpoint Snap Tab Collar B...


In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

#model = implicit.als.AlternatingLeastSquares(factors=128)
model = implicit.bpr.BayesianPersonalizedRanking(factors=128)

model.fit(ratings) #rating  =  sparse matrix

# recommend items for a user
user_items = ratings.T.tocsr()
recommendations = model.recommend(1, user_items)

related = model.similar_items(1)

#print(recommendations)
#print(related)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
n=len(related)
l=[]
for i in range(n):
    s=[]
    for j in num_to_asin_dict.keys():
        if i==j:
            s.append(num_to_asin_dict[j])
            s.append(related[i][1])
    l.append(s)
result = pd.DataFrame(l)
result.rename({0:'asin',1:'accuracy'}, axis=1,inplace=True)
user_recommended = pd.merge(result,df1, on='asin', how='inner')

user_recommended

,asin,accuracy,title
0,7106116521,1.000000,Milliongadgets(TM) Earring Safety Backs For Fi...
1,B00007GDFV,0.327374,Buxton Heiress Pik-Me-Up Framed Case
2,B00007GDFV,0.327374,Buxton Heiress Pik-Me-Up Framed Case
3,B00008JOQI,0.294670,Paul Fredrick Men's Pinpoint Button Down Colla...
4,B00008JOQI,0.294670,Paul Fredrick Men's Pinpoint Button Down Colla...
5,B00008JQF0,0.289063,Paul Fredrick Men's Pinpoint Straight Collar B...
6,B00008JQF0,0.289063,Paul Fredrick Men's Pinpoint Straight Collar B...
7,B00008JVTT,0.281424,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
8,B00008JVTT,0.281424,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
9,B00008JPRZ,0.278809,Paul Fredrick Men's Pinpoint Snap Tab Collar B...


In [ ]:
n=len(recommendations)
l=[]
for i in range(n):
    s=[]
    for j in num_to_asin_dict.keys():
        if i==j:
            s.append(num_to_asin_dict[j])
            s.append(recommendations[i][1])
    l.append(s)
result = pd.DataFrame(l)
result.rename({0:'asin',1:'accuracy'}, axis=1,inplace=True)
user_recommended = pd.merge(result,df1, on='asin', how='inner')

user_recommended

,asin,accuracy,title
0,7106116521,0.003238,Milliongadgets(TM) Earring Safety Backs For Fi...
1,B00007GDFV,0.003220,Buxton Heiress Pik-Me-Up Framed Case
2,B00007GDFV,0.003220,Buxton Heiress Pik-Me-Up Framed Case
3,B00008JOQI,0.003210,Paul Fredrick Men's Pinpoint Button Down Colla...
4,B00008JOQI,0.003210,Paul Fredrick Men's Pinpoint Button Down Colla...
5,B00008JQF0,0.003182,Paul Fredrick Men's Pinpoint Straight Collar B...
6,B00008JQF0,0.003182,Paul Fredrick Men's Pinpoint Straight Collar B...
7,B00008JVTT,0.003181,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
8,B00008JVTT,0.003181,Paul Fredrick Men's Pinpoint Snap Tab Collar F...
9,B00008JPRZ,0.003181,Paul Fredrick Men's Pinpoint Snap Tab Collar B...


#Recommendation by category

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error
import gzip
import json
from copy import deepcopy

##Read Data

In [ ]:
data1 = []
with gzip.open(r'/content/meta_Grocery_and_Gourmet_Food.json.gz') as f:
    for l in f:
        data1.append(json.loads(l.strip()))
print(len(data1))

In [ ]:
df_meta = pd.DataFrame.from_dict(data1)
df_meta.head(5)

In [ ]:
data2 = []
with gzip.open(r'/content/Grocery_and_Gourmet_Food.json.gz') as f:
    for l in f:
        data2.append(json.loads(l.strip()))
print(len(data2))
print(data2[0])

In [ ]:
df_review = pd.DataFrame.from_dict(data2)
df_review.head(5)

##meta data prepare

In [ ]:
df_meta['main_cat'].value_counts()

NameError: ignored

In [ ]:

meta_copy=deepcopy(df_meta)
meta_copy=meta_copy[['asin','category']]
meta_list=meta_copy.values.tolist()


In [ ]:
category_list=[]
for i in range(len(meta_list)):
    s=[]
    s.append(meta_list[i][0]) 
    s.append(meta_list[i][1][1])
    category_list.append(s)
df_cat_asin=pd.DataFrame(category_list)
df_cat_asin.rename({0:'asin',1:'category_type'}, axis=1,inplace=True)
df_cat_asin

In [ ]:
df_inner_meta = pd.merge(df_meta,df_cat_asin, on='asin', how='inner')

df_inner_meta.head(5)

In [ ]:
df_inner_meta['category_type'].value_counts()

In [ ]:
df_inner_meta_copy=deepcopy(df_inner_meta)

In [ ]:
df_inner_meta.drop([ 'brand', 'feature', 'rank', 'date', 'imageURL',
       'imageURLHighRes', 'description', 'price', 'also_view', 'also_buy'
       'fit', 'details', 'similar_item', 'tech1','tech2','main_cat','category'], axis=1, inplace=True)

df_inner_meta.head(3)

##Review data prepare


In [ ]:
df_review.drop(['verified', 'reviewTime', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'], axis = 1, inplace = True)
df_review.head(5)

##Data Prepare for recommendation

In [ ]:
def data_prepare(df_rev,df_met,category):
    df_inner_data=pd.merge(df_rev,df_met, on='asin', how='inner')
    preprocessed = df_inner_data.values.tolist()
    temp_data=preprocessed.loc[preprocessed['categoty']==category]
    temp_list= temp_data.values.tolist()
    unique_asin = temp_data['asin'].unique()
    count = 0
    asin_to_num_dict = {}
    num_to_asin_dict = {}
    for i in unique_asin:
        asin_to_num_dict[i] = count
        num_to_asin_dict[count] = i
        count+=1
    l=[]
    for i in asin_to_num_dict.keys():
        l.append([i,asin_to_num_dict[i]])
        dff=pd.DataFrame(l)
    dff.rename({1:'asin2num',0:'asin'}, axis=1,inplace=True)
    df_inner = pd.merge(temp_data,dff, on='asin', how='inner')
    unique_reviewer = new_df['reviewerID'].unique()
    count2 = 0
    reviewer_to_num_dict = {}
    num_to_reviewer_dict = {}
    for i in unique_reviewer:
        reviewer_to_num_dict[i] = count2
        num_to_reviewer_dict[count2] = i
        count2+=1
    l1=[]
    for i in reviewer_to_num_dict.keys():
        l1.append([i,reviewer_to_num_dict[i]])
    dff2=pd.DataFrame(l1)
    dff2.rename({1:'reviewer2num',0:'reviewerID'}, axis=1,inplace=True)
    df_inner2 = pd.merge(df_inner,dff2, on='reviewerID', how='inner')
    ##df_final=df_inner2.drop(['reviewerID','asin','category_type','title'],axis=1)
    ##selected_columns = df_final[["reviewer2num","asin2num","overall"]]
    ##new_df_final = selected_columns.copy()
    #print(new_df_final)
    return(df_inner1)



In [ ]:
def sparse(data):
    df_final=data.drop(['reviewerID','asin','category_type','title'],axis=1)
    selected_columns = df_final[["reviewer2num","asin2num","overall"]]
    new_df_final = selected_columns.copy()
    new_df_final.drop_duplicates(subset=["reviewer2num","asin2num"], keep='first', inplace=True)
    df_mat = df_l2.pivot(index='reviewer2num', columns='asin2num', values='overall')
    df_mat.fillna(0)
    from scipy.sparse import csr_matrix
    ratings=csr_matrix(df_mat2)
    return(ratings)

##recommendation

In [ ]:
pip install implicit

In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

In [ ]:
def recommendation(id,category,df_rev,df_met):
    unique_reviewer = new_df['reviewerID'].unique()
    count = 0
    #id to seqential
    reviewer_to_num_dict = {}
    num_to_reviewer_dict = {}
    for i in unique_reviewer:
        reviewer_to_num_dict[i] = count
        num_to_reviewer_dict[count] = i
        count+=1
    for i in reviewer_to_num_dict.keys():
        if i==id:
            seq_id=reviewer_to_num_dict[i]  #doubt
            break
    #data 
    data1=data_prepare(df_rev,df_met,category)
    ratings=sparse(data1)

    if id in data1['reviewerID']:


        model = implicit.als.AlternatingLeastSquares(factors=128)
        #model = implicit.bpr.BayesianPersonalizedRanking(factors=128)

        model.fit(ratings) #rating  =  sparse matrix

        # recommend items for a user
        user_items = ratings.T.tocsr()
        recommendations = model.recommend(seq_id, user_items)

       #related = model.similar_items(1)
    #popularity based 
    else:
        data1=data1.drop['asin2num','reviewer2num']
        ratings = pd.DataFrame(data1.groupby('asin')['overall'].median())   ##
        ratings.rename(columns={"overall": "Rating_Mean"}, inplace=True)       ##
        recommendation=ratings[:5]


    print(recommendations)
#print(related)